# Team Justus Von Liebug

## Introduction

Study of minimum cost diet for diabetic undergraduate students at UC Berkeley.

We begin by installing material from lecture.

In [4]:
!pip install -r requirements.txt;
import numpy as np
import pandas as pd
import fooddatacentral as fdc

In [5]:
!gpg --batch --passphrase "importance variation" -d justus_von_liebig-069d38b622a8.json.gpg > justus_von_liebig-069d38b622a8.json

gpg: AES encrypted data
gpg: encrypted with 1 passphrase


User and API Key from USDA:

In [6]:
user = "ligon"

# API key for Gov; substitute your own!
apikey = "NVgmRkqQ7ikdgus7M2OTqZFlDXm67vdQ3aiP1GND"

# File with private keys for relevant service account to authenticate
# and access google spreadsheets
serviceacct = {'ligon':'students-9093fa174318.json'}

## Data Collection

Using previous purchases of our group members from receipts, we compiled a small database of groceries we assume to be available to a typical UC Berkeley undergraduate. This data was recorded and stores in a Google Sheet. The URL can be found here: https://docs.google.com/spreadsheets/d/1xnjnl42g_jYKdkOfW2OF7tBdGmuCAeJluT2O7fx1OQE/edit?usp=sharing

Information from the following categories were inputted:

- **Food:** Label used to identify food
- **Quantity:** Amount of food
- **Units:** Units in which amounts of this food is measured
- **Price:** Price for quantity of food
- **FDC Code** Key identifying food in the FoodDataCentral dataset. *See below.*

Food from our sample was manually matched to foods in the FDC database using the following code:

In [7]:
import fooddatacentral as fdc

# Search by typing the name of the food.
fdc.search(apikey,"green bell peppers")

,fdcId,description,lowercaseDescription,commonNames,additionalDescriptions,dataType,foodCode,publishedDate,allHighlightFields,score,foodNutrients,ndbNumber,scientificName
0,1103561,"Peppers, green, cooked","peppers, green, cooked",,NS as to color;with or without fat,Survey (FNDDS),75226020,2020-10-30,,503.43222,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,NaN
1,1104580,"Green pepper, cooked, as ingredient","green pepper, cooked, as ingredient",,,Survey (FNDDS),99997520,2020-10-30,,459.76486,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,NaN
2,1103371,"Pepper, sweet, green, raw","pepper, sweet, green, raw",,,Survey (FNDDS),75122100,2020-10-30,,459.76486,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,NaN
3,168577,"Peppers, chili, green, canned","peppers, chili, green, canned",,,SR Legacy,NaN,2019-04-01,,459.76486,"[{'nutrientId': 1062, 'nutrientName': 'Energy'...",11980,NaN
4,170427,"Peppers, sweet, green, raw","peppers, sweet, green, raw",,,SR Legacy,NaN,2019-04-01,,459.76486,"[{'nutrientId': 1186, 'nutrientName': 'Folic a...",11333,Capsicum annuum
5,170024,"Peppers, sweet, green, sauteed","peppers, sweet, green, sauteed",,,SR Legacy,NaN,2019-04-01,,459.76486,"[{'nutrientId': 1062, 'nutrientName': 'Energy'...",11339,NaN
6,170497,"Peppers, hot chili, green, raw","peppers, hot chili, green, raw",,,SR Legacy,NaN,2019-04-01,,423.08096,"[{'nutrientId': 1186, 'nutrientName': 'Folic a...",11670,Capsicum frutescens
7,170336,"TACO BELL, Nachos","taco bell, nachos",,,SR Legacy,NaN,2019-04-01,,396.19614,"[{'nutrientId': 1267, 'nutrientName': '20:0', ...",21268,NaN
8,170429,"Peppers, sweet, green, canned, solids and liquids","peppers, sweet, green, canned, solids and liquids",,,SR Legacy,NaN,2019-04-01,,391.82935,"[{'nutrientId': 1062, 'nutrientName': 'Energy'...",11335,NaN
9,170022,"Peppers, sweet, green, frozen, chopped, unprep...","peppers, sweet, green, frozen, chopped, unprep...",,,SR Legacy,NaN,2019-04-01,,391.82935,"[{'nutrientId': 1062, 'nutrientName': 'Energy'...",11337,NaN


## Loading Groceries Data from Google Sheets

The following loads data inputted in Google Sheets and creates a data frame.

Creating dataframe containing 'Groceries' data:

In [8]:
df = pd.read_csv('groceries.csv')
df = df[df['Units'].isin(['g', 'lb'])]

As different items in the 'groceries' dataset are listed at different price points for different quantities and units, we normalize the data to appear similar for comparison and analysis. The FDC database has a preference for items in terms of hundreds of grams (hectograms) or hundreds of milliliters (deciliters).

The units function is used to convert all foods to either deciliters or hectograms, to match FDC database:

In [9]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food')['FDC Price'].min()

Prices

Food
Asparagus                                          0.4387199017479063 / hectogram
Broccoli Florets                                   1.9819557370420493 / hectogram
California sweet Oranges                          0.23442487212325314 / hectogram
Challenge unsalted butter                          1.3205689504874165 / hectogram
Cream Cheese                                        0.668000654420179 / hectogram
Crushed red pepper                                 1.5322127221848991 / hectogram
Dole, Organic Bananas, 2 lb Bag                   0.18629061154622154 / hectogram
Fair trade bananas                                0.26235009200000425 / hectogram
Fish Filet frozen                                  1.0273541417815295 / hectogram
Fish cake Frozen                                    1.100106688302539 / hectogram
Lean ground turkey                                 0.6795386195189564 / hectogram
Lemons, Small, 2 lb bag                           0.26455471462185304 / hectogram
Napa cabbag

## Dietary Requirements for Diabetes

Although there is no one-size-fits-all approach that works for every individual with diabetes, research reveals a general trend  in diets for those with diabetes to manage blood sugar levels and ultimately maintain long-term health. The American Diabetes Association (ADA) recommends a diet that does the following for diabetics:

1. Reduces carbohydrates intake
2. Limits sodium to less than 2,300 mg a day
3. Incorporates more non-starchy vegetables

We reference US government recommendations available at https://www.dietaryguidelines.gov/sites/default/files/2019-05/2015-2020_Dietary_Guidelines.pdf according to age and sex. We then make additional adjustments to these recommendations for someone with diabetes.

**Sources:**
https://www.diabetes.org/healthy-living/recipes-nutrition
https://www.diabetesfoodhub.org/articles/what-is-the-diabetes-plate-method.html#:~:text=The%20Diabetes%20Plate%20Method%20is,you%20need%20is%20a%20plate!
https://www.mayoclinic.org/diseases-conditions/diabetes/in-depth/diabetes-diet/art-20044295

## 1. A Low-Carbohydrate Diet

We define a low-carbohydrate diet to be half of the recommended intake of 140 grams of carbohydrates a day (exlcuding fiber). In this case, carbohydrates should not exceed 70 grams per day.

In [10]:
# Load minimum dietary requirements

bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:, 2:]
bmin

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [11]:
# Set new requirement for a low-carb diet

bmin.loc['Carbohydrate, by difference'] = 70
bmin

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


## 2. Limit Sodium Intake

Sodium intake should not exceed more than 2,300 grams per day...for anyone! The US Dietary guidelines outlines maximum sodium intake for each demographic by age and sex.

In [12]:
# Load maximum dietary requirements

bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
bmax

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
"Sodium, Na",1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300


In [13]:
# Concatenate updated minimum and maximum dietary requirements adjusted for individuals with diabetes

b = pd.concat([bmin, bmax])
b

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


## Dietary Reference Intake (DRI) for a Select Group

Find out the DRI for a select group by age and sex.

In [14]:
# Creates a function that takes two integer parameters: age and sex

def DRI(age, sex):
    if sex == "M":
        if age <=3:
            return b.loc[:, 'C 1-3']
        if age>=4 and age<=8:
            return b.loc[:, 'M 4-8']
        if age>=9 and age<=13:
            return b.loc[:, 'M 9-13']
        if age>= 14 and age<=18:
            return b.loc[:,'M 14-18']
        if age>=19 and age <= 30: 
            return b.loc[:, 'M 19-30']
        if age>=31 and age<=50:
            return b.loc[:, 'M 31-50']
        if age>=51:
            return b.loc[:, 'M 51+']
    else:
        if age <=3:
            return b.loc[:, 'C 1-3']
        if age>=4 and age<=8:
            return b.loc[:, 'F 4-8']
        if age>=9 and age<=13:
            return b.loc[:, 'F 9-13']
        if age>= 14 and age<=18:
            return b.loc[:,'F 14-18']
        if age>=19 and age <= 30: 
            return b.loc[:, 'F 19-30']
        if age>=31 and age<=50:
            return b.loc[:, 'F 31-50']
        if age>=51:
            return b.loc[:, 'F 51+']

In [15]:
# Choose an age and sex to get DRI

DRI(25, 'M')

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference         70.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Sodium, Na                        2300.0
Name: M 19-30, dtype: float64

## Mapping Food to Nutrients

Using our dataset of groceries, we look up nutritional values of foods that are accessible and consumed by UC Berkeley undergraduate students. 

In [16]:
# Creates a new dataframe detailing nutrients of items in 'groceries' dataset

import fooddatacentral as fdc

D = {}
for food in df['Food'].tolist():
    try:
        fdc_id = int((df[df['Food'] == food].FDC).iloc[0])
        D[food] = fdc.nutrients(apikey, fdc_id).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food, FDC))

FoodNutrients = pd.DataFrame(D,dtype=float)
FoodNutrients

,Ottogi Gold Mayonnaise,Sajo Light Tuna,Sanuki Udon,Pork belly skin on thick,pork belly skinless thin,Napa cabbage,Crushed red pepper,Salted shrimp,Rotisserie Chicken,Broccoli Florets,...,green bell peppers,Natural Fresh Chicken Thighs | Antibiotic Free,"Lemons, Small, 2 lb bag","Dole, Organic Bananas, 2 lb Bag","Wonderful Halos Mandarins, 3lb Bag",chicken breast,Fish Filet frozen,California sweet Oranges,Fish cake Frozen,Asparagus
10:0,0.000,NaN,NaN,0.04,0.04,NaN,NaN,NaN,NaN,NaN,...,0.00,NaN,0.000,NaN,NaN,NaN,0.001,0.00,0.059,NaN
11:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN
12:0,0.000,NaN,NaN,0.06,0.06,NaN,NaN,NaN,NaN,NaN,...,0.00,NaN,0.000,NaN,NaN,NaN,0.001,0.00,0.067,NaN
14:0,0.055,0.014,NaN,0.70,0.70,NaN,NaN,NaN,NaN,NaN,...,0.00,NaN,0.001,NaN,NaN,NaN,0.004,0.00,0.240,NaN
14:1 c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitamin K (Dihydrophylloquinone),NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vitamin K (phylloquinone),163.000,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,...,7.40,NaN,0.000,NaN,NaN,NaN,NaN,0.00,6.300,NaN
Vitamins and Other Components,0.000,0.000,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,...,0.00,NaN,0.000,NaN,NaN,NaN,0.000,0.00,0.000,NaN
Water,21.650,63.160,NaN,36.74,36.74,NaN,NaN,NaN,NaN,NaN,...,93.89,NaN,88.980,NaN,NaN,NaN,83.400,86.75,61.420,NaN


## Putting it together
*Taken from class material:*

Here we take the different pieces of the puzzle we’ve developed and put them together in the form of a linear program we can solve. Recall that the mathematical problem we’re trying to solve is$$
    \min_x p'x
$$such that$$
     Ax \geq b
$$If we buy a bag of groceries with quantities given by $x$, the total cost of the bag of groceries is the inner product of prices and quantities. Since we’ve converted our units above, this gives us a vector of prices where quantities are all in 100 g or ml units.

The following code block defines a function

In [17]:
from  scipy.optimize import linprog as lp
import numpy as np

def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    p = Prices.apply(lambda x:x.magnitude).dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = list(set(p.index.tolist()).intersection(FoodNutrients.columns.tolist()))
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[diet_min.index]

    Amax = Aall.loc[diet_max.index]

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,-Amax])

    b = pd.concat([diet_min,-diet_max]) # Note sign change for max constraints

    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    result.diet = pd.Series(result.x,index=p.index)

    return result

## Using =solve_subsistence_problem= to analyze diet
Let’s choose a particular group (type of person with particular dietary requirements) and solve the subsistence problem for them:

In [18]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,bmin[group],bmax[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $15.01 per day.


Diet (in 100s of grams or milliliters):
Food
green bell peppers          80.433438
Fish Filet frozen            1.246296
pork belly skinless thin     0.141998
cheddar cheese               0.253846
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                           2400.000044          2400.0
Protein                            97.159809            56.0
Fiber, total dietary              136.736848            33.6
Folate, DFE                       804.476397           400.0
Calcium, Ca                      1000.000015          1000.0
Carbohydrate, by difference       374.117403            70.0
Iron, Fe                           27.633080             8.0
Magnesium, Mg                     831.199223           400.0
Niacin                             43.455471            16.0
Phosphorus, P   

## Effects of Price Changes on Subsistence Diet Cost

In [19]:
import cufflinks as cf
cf.go_offline()

scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

cost0 = solve_subsistence_problem(FoodNutrients,Prices,bmin[group],bmax[group],tol=tol).fun

Price_response={}
for s in scale:
    cost = {}
    for i,p in enumerate(Prices):
        my_p = Prices.copy()
        my_p[i] = p*s
        result = solve_subsistence_problem(FoodNutrients,my_p, bmin[group], bmax[group],tol=tol)
        cost[Prices.index[i]] = np.log(result.fun/cost0)
    Price_response[np.log(s)] = cost

Price_response = pd.DataFrame(Price_response).T
Price_response.iplot(xTitle='log price change',yTitle='log cost change')

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning:

The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.

Error importing optional module geopandas
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/_plotly_utils/optional_imports.py", line 30, in get_module
    return import_module(name)
  File "/opt/conda/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>